# Imports

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

# Data Reading and Preparation

In [7]:
data = pd.read_csv('InputTrain.csv')
train  = pd.read_csv('InputTrain.csv')
test = pd.read_csv('InputTest.csv')
WM = pd.read_csv('StepTwo_LabelTrain_WashingMachine.csv')
DW = pd.read_csv('StepTwo_LabelTrain_Dishwasher.csv')
TD = pd.read_csv('StepTwo_LabelTrain_TumbleDryer.csv')
MW = pd.read_csv('StepTwo_LabelTrain_Microwave.csv')
K= pd.read_csv('StepTwo_LabelTrain_Kettle.csv')

In [8]:
data = data.set_index('Index')
train = train.set_index('Index')
test =  test.set_index('Index')
WM = WM.set_index('Index')
DW = DW.set_index('Index')
TD = TD.set_index('Index')
MW = MW.set_index('Index')
K = K.set_index('Index')

In [9]:
X = train.drop('House_id', axis=1)
Y_WM = WM.drop('House_id',axis=1)
Y_DW = DW.drop('House_id',axis=1)
Y_TD = TD.drop('House_id',axis=1)
Y_MW = MW.drop('House_id',axis=1)
Y_K = K.drop('House_id',axis=1)
test_X = test.drop("House_id",axis=1)

In [10]:
X.insert(len(X.columns), "min", data.drop("House_id",axis=1).min(axis=1))
X.insert(len(X.columns), "max", data.drop("House_id",axis=1).max(axis=1))
X.insert(len(X.columns), "std", data.drop("House_id",axis=1).std(axis=1))
X.insert(len(X.columns), "var", data.drop("House_id",axis=1).var(axis=1))
X

,TimeStep_0,TimeStep_1,TimeStep_2,TimeStep_3,TimeStep_4,TimeStep_5,TimeStep_6,TimeStep_7,TimeStep_8,TimeStep_9,...,TimeStep_2155,TimeStep_2156,TimeStep_2157,TimeStep_2158,TimeStep_2159,min,max,std,var,mean
Index,,,,,,,,,,,,,,,,,,,,,
0,180.000000,180.0,180.0,181.0,180.0,180.0,180.000000,181.0,180.0,181.0,...,238.333333,236.0,234.0,234.0,235.0,179.000000,3566.0,346.031985,119738.134353,274.770139
1,2437.000000,2426.0,2148.0,645.0,642.0,642.0,2256.666667,2436.0,2436.0,2212.0,...,236.000000,234.0,234.0,230.0,233.0,180.000000,5444.0,517.594892,267904.472169,464.613580
2,232.000000,232.0,232.5,233.0,233.0,234.0,233.000000,233.0,232.0,234.0,...,181.500000,181.0,180.0,180.0,180.0,178.666667,553.0,52.104176,2714.845137,212.391667
3,180.333333,181.0,180.0,184.0,181.0,180.0,183.000000,184.0,184.0,183.0,...,349.000000,348.5,340.0,346.0,344.0,151.000000,2701.0,267.951248,71797.871555,302.598225
4,344.000000,341.0,341.0,327.0,327.0,318.0,318.000000,313.0,313.0,315.0,...,235.000000,233.0,233.0,235.0,233.0,153.000000,399.0,37.034982,1371.589907,223.516049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10416,275.000000,272.0,270.0,275.0,269.0,267.0,269.000000,268.5,270.0,285.0,...,280.000000,281.0,280.0,280.0,281.0,230.000000,4092.0,282.094349,79577.221781,345.242593
10417,279.000000,283.5,281.0,281.0,277.0,281.5,283.000000,284.5,281.0,281.0,...,230.000000,230.0,230.5,231.0,231.0,228.000000,737.0,61.822732,3822.050191,293.144290
10418,229.000000,229.0,231.0,231.0,230.0,230.0,229.500000,230.0,231.0,231.0,...,231.000000,231.5,231.0,230.5,231.0,228.000000,1156.0,40.864152,1669.878893,256.794059


# XGBoost Algorithm

### Washing Machine

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_WM, random_state=1)

In [12]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

'eta' : 0.2,'max_depth': 4,"alpha":2,"subsample":0.829,'min_child_weight':12,'gamma':0.1,'max_depth': 4,

eta : 0.2 -----> eval-rmse:0.15345
 'max_depth': 3 ----> eval-rmse:0.15435
 "alpha":1,'max_depth': 4,'eta' : 0.2 -------> eval-rmse:0.15174
 "alpha":2,'max_depth': 4,'eta' : 0.2 -------> eval-rmse:0.15143

In [13]:
params = {'alpha':2,'max_depth': 4,'eta' : 0.2,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 100
bst_WM = xgb.train(params, dtrain_reg, n, evals = evalist,early_stopping_rounds=1)

[0]	train-rmse:0.40958	eval-rmse:0.41115
[1]	train-rmse:0.33871	eval-rmse:0.34220
[2]	train-rmse:0.28356	eval-rmse:0.28936
[3]	train-rmse:0.24105	eval-rmse:0.24952
[4]	train-rmse:0.20867	eval-rmse:0.22009
[5]	train-rmse:0.18429	eval-rmse:0.19881
[6]	train-rmse:0.16617	eval-rmse:0.18379
[7]	train-rmse:0.15281	eval-rmse:0.17338
[8]	train-rmse:0.14297	eval-rmse:0.16630
[9]	train-rmse:0.13572	eval-rmse:0.16154
[10]	train-rmse:0.13028	eval-rmse:0.15834
[11]	train-rmse:0.12607	eval-rmse:0.15617
[12]	train-rmse:0.12273	eval-rmse:0.15470
[13]	train-rmse:0.12001	eval-rmse:0.15368
[14]	train-rmse:0.11774	eval-rmse:0.15296
[15]	train-rmse:0.11578	eval-rmse:0.15245
[16]	train-rmse:0.11404	eval-rmse:0.15207
[17]	train-rmse:0.11244	eval-rmse:0.15179
[18]	train-rmse:0.11096	eval-rmse:0.15156
[19]	train-rmse:0.10956	eval-rmse:0.15137
[20]	train-rmse:0.10823	eval-rmse:0.15121
[21]	train-rmse:0.10697	eval-rmse:0.15108
[22]	train-rmse:0.10576	eval-rmse:0.15097
[23]	train-rmse:0.10458	eval-rmse:0.15086
[2

In [14]:
bst_WM.save_model("WM_model")

### Dishwasher

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_DW, random_state=2)

In [16]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [17]:
params = {'alpha':2,'max_depth': 4,'eta' : 0.2 ,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 100
bst_DW = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

C:\Users\moham\AppData\Roaming\Python\Python310\site-packages\xgboost\core.py:617: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


[0]	train-rmse:0.40575	eval-rmse:0.40652
[1]	train-rmse:0.33126	eval-rmse:0.33300
[2]	train-rmse:0.27263	eval-rmse:0.27563
[3]	train-rmse:0.22675	eval-rmse:0.23127
[4]	train-rmse:0.19112	eval-rmse:0.19748
[5]	train-rmse:0.16374	eval-rmse:0.17216
[6]	train-rmse:0.14294	eval-rmse:0.15359
[7]	train-rmse:0.12728	eval-rmse:0.14023
[8]	train-rmse:0.11553	eval-rmse:0.13080
[9]	train-rmse:0.10672	eval-rmse:0.12424
[10]	train-rmse:0.10001	eval-rmse:0.11966
[11]	train-rmse:0.09479	eval-rmse:0.11650
[12]	train-rmse:0.09070	eval-rmse:0.11428
[13]	train-rmse:0.08742	eval-rmse:0.11271
[14]	train-rmse:0.08477	eval-rmse:0.11161
[15]	train-rmse:0.08258	eval-rmse:0.11084
[16]	train-rmse:0.08074	eval-rmse:0.11027
[17]	train-rmse:0.07913	eval-rmse:0.10986
[18]	train-rmse:0.07769	eval-rmse:0.10954
[19]	train-rmse:0.07640	eval-rmse:0.10928
[20]	train-rmse:0.07520	eval-rmse:0.10907
[21]	train-rmse:0.07412	eval-rmse:0.10890
[22]	train-rmse:0.07310	eval-rmse:0.10876
[23]	train-rmse:0.07214	eval-rmse:0.10863
[2

In [18]:
bst_DW.save_model("DW_model")

### Tumble Dryer

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_TD, random_state=3)

In [20]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [21]:
params = {'alpha':2,'max_depth': 4,'eta' : 0.2,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 100
bst_TD = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

[0]	train-rmse:0.40172	eval-rmse:0.40224
[1]	train-rmse:0.32330	eval-rmse:0.32446
[2]	train-rmse:0.26078	eval-rmse:0.26278
[3]	train-rmse:0.21104	eval-rmse:0.21407
[4]	train-rmse:0.17155	eval-rmse:0.17586
[5]	train-rmse:0.14032	eval-rmse:0.14615
[6]	train-rmse:0.11575	eval-rmse:0.12336
[7]	train-rmse:0.09653	eval-rmse:0.10614
[8]	train-rmse:0.08161	eval-rmse:0.09340
[9]	train-rmse:0.07014	eval-rmse:0.08418
[10]	train-rmse:0.06137	eval-rmse:0.07766
[11]	train-rmse:0.05469	eval-rmse:0.07311
[12]	train-rmse:0.04966	eval-rmse:0.07000
[13]	train-rmse:0.04586	eval-rmse:0.06791
[14]	train-rmse:0.04304	eval-rmse:0.06650
[15]	train-rmse:0.04093	eval-rmse:0.06556
[16]	train-rmse:0.03932	eval-rmse:0.06493
[17]	train-rmse:0.03804	eval-rmse:0.06451
[18]	train-rmse:0.03697	eval-rmse:0.06422
[19]	train-rmse:0.03606	eval-rmse:0.06401
[20]	train-rmse:0.03525	eval-rmse:0.06387
[21]	train-rmse:0.03453	eval-rmse:0.06377
[22]	train-rmse:0.03385	eval-rmse:0.06368
[23]	train-rmse:0.03323	eval-rmse:0.06362
[2

In [22]:
bst_TD.save_model("TD_model")

### Microwave

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_MW, random_state=4)

In [24]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [25]:
params = {'alpha':2,'max_depth': 4,'eta' : 0.2,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 100
bst_MW = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

[0]	train-rmse:0.40068	eval-rmse:0.40067
[1]	train-rmse:0.32136	eval-rmse:0.32135
[2]	train-rmse:0.25809	eval-rmse:0.25809
[3]	train-rmse:0.20769	eval-rmse:0.20772
[4]	train-rmse:0.16763	eval-rmse:0.16770
[5]	train-rmse:0.13588	eval-rmse:0.13603
[6]	train-rmse:0.11082	eval-rmse:0.11109
[7]	train-rmse:0.09114	eval-rmse:0.09160
[8]	train-rmse:0.07583	eval-rmse:0.07656
[9]	train-rmse:0.06402	eval-rmse:0.06510
[10]	train-rmse:0.05498	eval-rmse:0.05654
[11]	train-rmse:0.04814	eval-rmse:0.05028
[12]	train-rmse:0.04298	eval-rmse:0.04578
[13]	train-rmse:0.03909	eval-rmse:0.04262
[14]	train-rmse:0.03611	eval-rmse:0.04045
[15]	train-rmse:0.03379	eval-rmse:0.03897
[16]	train-rmse:0.03198	eval-rmse:0.03799
[17]	train-rmse:0.03048	eval-rmse:0.03732
[18]	train-rmse:0.02926	eval-rmse:0.03689
[19]	train-rmse:0.02824	eval-rmse:0.03660
[20]	train-rmse:0.02743	eval-rmse:0.03641
[21]	train-rmse:0.02676	eval-rmse:0.03629
[22]	train-rmse:0.02621	eval-rmse:0.03621
[23]	train-rmse:0.02574	eval-rmse:0.03615
[2

In [26]:
bst_MW.save_model("MW_model")

### Kettle

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_K, random_state=5)

In [28]:
dtrain_reg = xgb.DMatrix(X_train, y_train)

dtest_reg = xgb.DMatrix(X_test, y_test)

In [29]:
params = {'alpha':2,'max_depth': 4,'eta' : 0.2,"objective": "reg:squarederror", "tree_method": "gpu_hist"}
evalist = [(dtrain_reg, 'train'), (dtest_reg, 'eval')]
n = 100
bst_K = xgb.train(params, dtrain_reg, n, evalist,early_stopping_rounds=10)

[0]	train-rmse:0.40137	eval-rmse:0.40156
[1]	train-rmse:0.32265	eval-rmse:0.32314
[2]	train-rmse:0.25987	eval-rmse:0.26078
[3]	train-rmse:0.20989	eval-rmse:0.21134
[4]	train-rmse:0.17017	eval-rmse:0.17232
[5]	train-rmse:0.13873	eval-rmse:0.14175
[6]	train-rmse:0.11395	eval-rmse:0.11802
[7]	train-rmse:0.09448	eval-rmse:0.09982
[8]	train-rmse:0.07936	eval-rmse:0.08611
[9]	train-rmse:0.06766	eval-rmse:0.07596
[10]	train-rmse:0.05872	eval-rmse:0.06862
[11]	train-rmse:0.05195	eval-rmse:0.06341
[12]	train-rmse:0.04684	eval-rmse:0.05978
[13]	train-rmse:0.04294	eval-rmse:0.05726
[14]	train-rmse:0.03999	eval-rmse:0.05555
[15]	train-rmse:0.03770	eval-rmse:0.05437
[16]	train-rmse:0.03589	eval-rmse:0.05355
[17]	train-rmse:0.03443	eval-rmse:0.05300
[18]	train-rmse:0.03325	eval-rmse:0.05261
[19]	train-rmse:0.03229	eval-rmse:0.05235
[20]	train-rmse:0.03150	eval-rmse:0.05217
[21]	train-rmse:0.03081	eval-rmse:0.05203
[22]	train-rmse:0.03018	eval-rmse:0.05193
[23]	train-rmse:0.02959	eval-rmse:0.05185
[2

In [30]:
bst_K.save_model("K_model")